# Fundamentals of Bioimage Analysis: I/O and Processing

## Course Introduction

Welcome to the foundational notebook of the P2N2025 bioimage analysis course! This notebook will teach you the essential skills for working with microscopy data using Python and open-source tools.

## Learning Objectives

By completing this notebook, you will be able to:

1. **Read and write** microscopy files in various formats (CZI, OME-TIFF, etc.)
2. **Navigate and understand** image metadata including pixel sizes and channel information
3. **Visualize** multidimensional microscopy data using matplotlib and napari

## Course Structure

This notebook is divided into three main chapters:

### Chapter 1: Reading and Writing Files
- **Objective**: Master the fundamentals of microscopy file I/O
- **Key Skills**: Using bioio for unified file access, understanding metadata, handling different file formats
- **Take-Home**: Bioio provides a standardized way to access diverse microscopy formats

### Chapter 2: Visualization
- **Objective**: Learn effective strategies for visualizing multidimensional data
- **Key Skills**: 2D plotting with matplotlib, 3D visualization with napari, creating publication-quality figures
- **Take-Home**: Proper visualization is essential for understanding your data and debugging analysis pipelines

## Why This Matters

Modern microscopy generates complex, multidimensional datasets that require computational tools to extract meaningful information. This notebook teaches you the fundamental skills to start with reading and writing files, and basic visualization.

**Let's begin!**


In [1]:
!which python

/usr/local/bin/python


# Library installations on Google Colab
When using Google Colab, we have to make sure that all the libraries that we will use are available in its runtime environment.
Unlike local Python environments that we can reuse after a session, Google Colab will reset the environment after a certain amount of time.
Therefore, each time we start a new Google Colab session, we have to install the libraries in the runtime environment. Luckily, Google Colab already comes with many libraries pre-installed. So, we only have to install the libraries that are not pre-installed.

The following code cell will install all required libraries for this notebook.

In [ ]:
# Pre-installed libraries (will be skipped automatically)
!pip install numpy
!pip install matplotlib

# To install
!pip install bioio==3.0.0 bioio-czi==2.4.0 bioio-ome-tiff==1.4.0
!pip install "napari[pyqt6]"

# Chapter 1: Reading and Writing Files
In this chapter, we will learn how to read and write microscopy files in various formats. 
Hereto, we will use the convenient `bioio` library that will handle a lot of the heavy lifting for us. 
It provides a unified interface for reading and writing microscopy files in various formats.

## 1.1. Reading Files

**Importing the `bioio` library**

We will start by reading files using the `bioio` library. The following code cell will import all the elements from `bioio` that we will use to read files in this notebook.

In [1]:
# Bioio imports
from bioio import BioImage
from bioio_ome_tiff.writers import OmeTiffWriter
import bioio_base as biob

**Reading a file**

We can now define a variable containing the path to the image file that we want to read. In this example, we have a .czi file, which is a file format used by Zeiss microscopes.

```python
image_path = "/content/sample_data/hela_prolongdiamond_dapi_tubulin-af488_mitochondria-af568_40x.czi"
```

We can then use the `BioImage` class to read the file. 

```python
img = BioImage(image_path)
```
A microscopy image file can contain multiple acquisitions, called scenes in `bioio`. We can specify which scene we want to read, but by default, the first scene will be read.

We can then print the id of the current scene using the `current_scene` attribute.

```python
print(img.current_scene)
```

We can also get a list of all the scenes in the file using the `scenes` attribute.

```python
print(img.scenes)
```

In [2]:
# Define the path to the image file
image_path = "/content/sample_data/hela_prolongdiamond_dapi_tubulin-af488_mitochondria-af568_40x.czi"

# Define a BioImage object
img = BioImage(image_path)  # selects the first scene found

# Get the id of the current operating scene
print(f'Current scene: {img.current_scene}')

# Get a list valid scene ids
print(f'Scenes: {img.scenes}')

0
('0', '1', '2', '3')


**Changing the scene**

To read another acquisition, a.k.a. scene, we can use the `set_scene` method.

This can be done by specifying the scene id, a string that is the name of the scene.
```python
img.set_scene("1")
```

We can also use the `set_scene` method with the scene index.

```python
img.set_scene(1)
```

In [3]:
# Change scene using name
img.set_scene("1")
# Or by scene index
img.set_scene(1)

**Accessing image metadata**

The `BioImage` class provides a lot of metadata about the image. We can access it as attributes of our `img` object.

In [4]:
# Access all the metadata
img.metadata  # returns the metadata object for this file format (XML, JSON, etc.)

<Dimensions [T: 1, C: 3, Z: 14, Y: 512, X: 512]>
TCZYX
512
(1, 3, 14, 512, 512)


In [ ]:
print(img.ome_metadata) # returns the OME-XML metadata as a string

In [ ]:
# Access specific metadata
print(f'Dimensions object: {img.dims}')  # returns a Dimensions object
print(f'Dimension order: {img.dims.order}')  # returns string "TCZYX"
print(f'Size of X dimension: {img.dims.X}')  # returns size of X dimension
print(f'Shape: {img.shape}')  # returns tuple of dimension sizes in TCZYX order

print(f'Channel names: {img.channel_names}')  # returns a list of string channel names found in the metadata
print(f'Z dimension pixel size: {img.physical_pixel_sizes.Z}')  # returns the Z dimension pixel size as found in the metadata
print(f'Y dimension pixel size: {img.physical_pixel_sizes.Y}')  # returns the Y dimension pixel size as found in the metadata
print(f'X dimension pixel size: {img.physical_pixel_sizes.X}')  # returns the X dimension pixel size as found in the metadata

## 1.2. Writing Files

We can also write files using the `bioio` library.

Hereto, we first define the output path, i.e. the path where we want to write the file to.
```python
output_path="/content/images/output.ome.tiff"
```

We can then use the `write_file` method to write the file.

```python
img.write_file("output.ome.tiff")
```

In this case, we are writing the image into an OME-TIFF file that will use the same metadata as the original file.


In [ ]:
# Define the output path
output_path="/content/images/output.ome.tiff"

# Write the file
img.write_file(output_path)

**Creating a new image file from scratch**

In most cases, we are not just opening an existing file and saving it again. 
Often, we have data stored in memory, like a numpy array, that we want to save as a file.
To store metadata together with the raw data, we can use the `bioio` library to create an image file from scratch.

As an example, we will extract the raw data from our image as a numpy array.

In [9]:
# Get the raw data as a numpy array
data = img.get_image_data("CZYX", T=0)  # returns 4D CZYX numpy array
print(f'Data type: {type(data)}')
print(f'Numpy array shape: {data.shape}')

We can now write the data to a new file. Hereto, we will import the `OmeTiffWriter` class from the `bioio` library that allows us to create a new OME-TIFF file from scratch.

We will also define the metadata that we want to stored in the new file. To do this in a standardized way, we will also import `bioio_base` that contains the base classes for the metadata. For convenience, we call it `biob`.

In [ ]:
# Import the OmeTiffWriter class
from bioio import OmeTiffWriter
import bioio_base as biob

# Get the raw data as a numpy array
data = img.get_image_data("CZYX", T=0)  # returns 4D CZYX numpy array
print(f'Numpy array shape: {data.shape}')

# Define the output path
output_path="/content/images/output.ome.tiff"

# Define the metadata
write_dim_order = "CZYX"  # required, the dimension order to write the data in
channel_names = ['Mitochondria', 'Tubulin', 'DAPI']
channel_colors = [(255,0,255), (0,255,0), (0,255,255)]  # Magenta/Green/Cyam, colors in RGB format
pixel_size = biob.types.PhysicalPixelSizes(
    img.physical_pixel_sizes.Z,
    img.physical_pixel_sizes.Y,
    img.physical_pixel_sizes.X
    )  # optional, Z, Y, X in microns

# Write the file
OmeTiffWriter.save(
        data,
        output_path,
        write_dim_order,
        channel_names=channel_names,
        channel_colors=channel_colors,
        physical_pixel_sizes=pixel_size,
    )

**Reading the stored file**

We can now read the file we just wrote using the `BioImage` class.

In [ ]:
img = BioImage(output_path)
print(img.ome_metadata)

# Chapter2: Visualisation
Now that we know how to read and write files, we can start visualising the data. In this chapter, we will explore various ways to visualize the data, from simple 2D images to 3D volumes.

For this, we will use a couple of new libraries:
- matplotlib: for simple 2D and 3D image visualisation and data plotting
- napari: for more advanced visualisation of images, volumes and time-series

## 2.1. Matplotlib for 2D image visualisation and data plotting

**2D image visualisation**

First, let's visualize the images in 2D using matplotlib.




In [10]:
# Define the image path
image_path = "/content/sample_data/hela_prolongdiamond_dapi_tubulin-af488_mitochondria-af568_40x.czi"

# Define a BioImage object
img = BioImage(image_path) 
print(f'Image shape: {img.shape}')
print(f'Image dimensions: {img.dims.order}')
print(f'Image physical pixel sizes: {img.physical_pixel_sizes}')
print(f'Image channel names: {img.channel_names}')

# Create variables from metadata
channel_names = img.channel_names
image_scale = [img.physical_pixel_sizes.Z, img.physical_pixel_sizes.Y, img.physical_pixel_sizes.X]

# Define colormaps to use for each channel
color_map_list = ['magenta', 'green', 'cyan']

# First, let use visualize the images in 2D using matplotlib
# The images are multi-dimensional. We will plot each channel in a separate subplot. 
# For each channel we will use a maximum intensity projection (MIP).
import numpy as np
import matplotlib.pyplot as plt

# Make sure matplotlib is displayed inline in the notebook
%matplotlib inline 

# Get the raw data as a numpy array
data = img.get_image_data("CZYX", T=0) # returns 4D CZYX numpy array
data_mip = np.max(data, axis=1) # returns 3D CYX numpy array (C=3, Y=1024, X=1024) containing MIP images.

# Create a figure with 3 subplots, 1 for each channel, arranged horizontally
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, channel in enumerate(channel_names):
    data_mip_channel = data_mip[i,:,:] # Get the MIP image for the i-th channel
    cmap = plt.get_cmap(color_map_list[i]) # Get the colormap for the i-th channel
    axes[i].imshow(data_mip_channel, cmap=cmap)
    axes[i].set_title(channel) # Set the title of the subplot to the channel name
    axes[i].axis('off') # Remove the axis ticks
plt.show()

**Data plotting**

Besides visualizing 2D images, we can also use matplotlib to plot data.

For example, we can plot the intensity distribution of the different channels.


In [ ]:
# Create a plot with the histograms of the different channels
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
for i, channel in enumerate(channel_names):
    data_mip_channel = data_mip[i,:,:] # Get the MIP image for the i-th channel
    cmap = plt.get_cmap(color_map_list[i]) # Get the colormap for the i-th channel

    # Plot the histogram of the channel
    ax.hist(
        data_mip_channel.flatten(), # Flatten the image to a 1D array
        bins=100, 
        color=cmap, 
        alpha=0.5
        )
    ax.set_title(channel) # Set the title of the subplot to the channel name
    ax.set_xlabel('Intensity') # Set the x-axis label to 'Intensity'
    ax.set_ylabel('Frequency') # Set the y-axis label to 'Frequency'
plt.show()

## 2.2. Visualzing volumes with Napari

Napari is a powerful tool for visualizing images, volumes and time-series. Here, we will use it to visuale the 3D volume of the image.

In [ ]:
import napari
from napari.utils import nbscreenshot

In [ ]:
# Create a napari Viewer
viewer = napari.Viewer(ndisplay=3) # use ndisplay=2 for 2D

for i, channel in enumerate(channel_names):
    viewer.add_image(
        img.get_image_data("ZYX", T=0, C=i),
        contrast_limits=[0, 4095],
        name=channel_names[i],
        scale=image_scale,
        colormap=color_map_list[i],
        blending='additive'
        )

# Rotate the image in 3D
viewer.camera.angles = (0, 15, 35)

nbscreenshot(viewer, canvas_only=True) # canvas_only=True removes the napari GUI


Available platform plugins are: minimalegl, wayland-egl, eglfs, xcb, linuxfb, wayland, offscreen, vkkhrdisplay, vnc, minimal.


Available platform plugins are: minimalegl, wayland-egl, eglfs, xcb, linuxfb, wayland, offscreen, vkkhrdisplay, vnc, minimal.

